In [99]:
import os
import re
import json
import pandas as pd
import re
from unidecode import unidecode

folderPathBaseIdealista = '..\\..\\Bases\\idealista\\202508'
folderPathBaseFinal = '..\\..\\Bases\\idealista\\base_processada\\base_imoveis.csv'

def listar_jsons(raiz):
    arquivos_json = []
    for pasta_atual, subpastas, arquivos in os.walk(raiz):
        for arquivo in arquivos:
            if arquivo.endswith('.json'):
                caminho_completo = os.path.join(pasta_atual, arquivo)
                arquivos_json.append(caminho_completo)
    return arquivos_json

def ler_todos_jsons_para_dataframe(raiz):
    lista_df = []
    arquivos_json = listar_jsons(raiz)
    for caminho in arquivos_json:
        try:
            df = pd.read_json(caminho)
            df['arquivo_origem'] = os.path.basename(caminho)  # nome do arquivo
            df['caminho_pasta'] = os.path.dirname(caminho)    # caminho da pasta
            lista_df.append(df)
        except Exception as e:
            print(f"Erro ao ler {caminho}: {e}")
    if lista_df:
        df_final = pd.concat(lista_df, ignore_index=True)
    else:
        df_final = pd.DataFrame()

    return df_final

def trata_imoveis(df):
    # Aplica ao DataFrame inteiro:
    df['municipio'] = df['arquivo_origem'].apply(extrair_municipio)
    df['caminho_pasta'] = df['caminho_pasta'].str.replace(r"^\.\.\\\.\.\\Bases\\idealista\\202508\\", "", regex=True)
    df['preco_mes'] = df['Preco'].apply(tratar_preco)
    df['numero_andar'] = df['pisoResumo'].apply(extrair_numero_andar)
    df['tem_elevador'] = df['pisoResumo'].apply(extrair_elevador)

    colunas = [
        'id',
        'Titulo',
        'detalhe do item',
        'Descricao do item',
        'estacionamento',
        'tipologia',
        'numero_andar',
        'tem_elevador',
        'pisoResumo',
        'tags',
        'Link',
        'areaBrutaM2',
        'Preco',
        'preco_mes',
        'precoOriginal',
        'descontoPercentual',
        'tempoDestaque',
        'imagens',
        'arquivo_origem',
        'caminho_pasta',
        'municipio'
    ]

    df = df[colunas]

    df = df.dropna(axis=1, how='all')

    return df

def extrair_municipio(nome_arquivo):
    # Pega tudo antes de "-Paginas"
    resultado = re.match(r"^(.*?)-Paginas", nome_arquivo)
    if resultado:
        return resultado.group(1)
    else:
        return nome_arquivo.replace('.json', '')  # fallback: tira o .json
    
def tratar_preco(preco):
    # Remove tudo que não é número, ponto ou vírgula
    preco_limpo = ''.join(c for c in preco if c.isdigit() or c in '.,')
    # Se tiver mais de um ponto, é separador de milhar
    if preco_limpo.count('.') > 0 and preco_limpo.count(',') == 0:
        # Ex: 5.000 -> 5000
        preco_limpo = preco_limpo.replace('.', '')
    # Se tiver vírgula, é decimal
    preco_limpo = preco_limpo.replace(',', '.')
    try:
        return float(preco_limpo)
    except:
        return None
    
def extrair_numero_andar(piso):
    if pd.isnull(piso):
        return None
    # Checa por 'Rés do chão' ou 'Cave'
    if "rés do chão" in piso.lower():
        return 0
    if "cave" in piso.lower():
        return -1
    
    # Novo padrão para 'Andar -2', 'Andar -1', etc
    match_andar_neg = re.search(r"andar\s*([-+]?\d+)", piso.lower())
    if match_andar_neg:
        return int(match_andar_neg.group(1))
    
    # Busca padrão de número
    match = re.search(r"(\d+)[ºo]? andar", piso.lower())
    if match:
        return int(match.group(1))
    return None

def extrair_elevador(piso):
    if pd.isnull(piso):
        return None
    piso_lower = piso.lower()
    if "com elevador" in piso_lower:
        return True
    if "sem elevador" in piso_lower:
        return False
    # Se só tem "elevador" e não especificou com/sem, marca como True
    if "elevador" in piso_lower:
        return True
    return None
    
def formatar_numericos(df):
        df = df.copy()  # Cria uma cópia, o original não será alterado!
        for col in df.select_dtypes(include='number').columns:
            df[col] = df[col].apply(lambda x: '{:,.2f}'.format(x).replace(',', 'X').replace('.', ',').replace('X', '.'))
        return df

def salva_base(df_imoveis):
    df_imoveis.to_csv(folderPathBaseFinal, index=False)


In [100]:
# Exemplo de uso:
df_total = ler_todos_jsons_para_dataframe(folderPathBaseIdealista)


In [101]:
df_imoveis = trata_imoveis(df_total)

In [102]:
base = formatar_numericos(df_imoveis)
salva_base(base)
